In [1]:
import numpy as np
import pandas as pd

In [150]:
clean = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/STA_208/data/clean.csv")

In [109]:
from Functions import Cleaning_Functions
fun = Cleaning_Functions()

In [110]:
market_data = fun.delete_id_columns(clean) #1
market_data, pred_market = fun.drop_response_rows_with_NAs(market_data, "Market_Orientation", "PPI_Likelihood") #2
market_data = fun.replace_NAN_with_na(market_data) #3
market_data = fun.entry_to_lowercase(market_data) #4
market_data = fun.remove_underscores_spaces(market_data) #5
market_data = fun.convert_to_categorical(market_data) #6
market_data = fun.impute_data(market_data)

In [111]:
pred_market.shape
market_data.YEAR = market_data.YEAR.astype('category')

In [112]:
def standarize_data(df):
    
    """
    Input: a dataset
    action: returns numeric column values scaled by mean and standard deviation
    """
    numeric_data = df.select_dtypes(include=['float64', 'int64'])
    for i in numeric_data.columns:
        df[i] = (df[i] - df[i].mean())/df[i].std()
    return df

In [113]:
market_data = standarize_data(market_data)

In [114]:
market_data.dtypes

crop_count                                       float64
crop_name_1                                     category
crop_harvest_1                                  category
crop_intercrop_1                                category
YEAR                                            category
Country                                         category
HHsizemembers                                    float64
HHsizeMAE                                        float64
HouseholdType                                   category
Head_EducationLevel                             category
LandOwned                                        float64
LandCultivated                                   float64
LivestockHoldings                                float64
NrofMonthsFoodInsecure                           float64
HFIAS_status                                    category
score_HDDS_GoodSeason                            float64
score_HDDS_farmbasedGoodSeason                   float64
score_HDDS_purchasedGoodSeason 

## Ridge Regression y = Market orientation

In [115]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

### 1. create dummy

In [116]:
def get_dummyXs_y(df, y_var):
    
    y = df[y_var]
    X  = df.drop(y_var, axis = 1)
    X_cat = X.select_dtypes(include = ["category", "O"])
    X_num = X.select_dtypes(include=['float64', 'int64'])
    
    X_cat_dummy = pd.get_dummies(X_cat)
    newX = pd.concat([X_num, X_cat_dummy], axis = 1)
    
    return newX, y

In [117]:
X_market_ridge = get_dummyXs_y(market_data, "Market_Orientation")[0]
y_market_ridge = get_dummyXs_y(market_data, "Market_Orientation")[1]

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X_market_ridge,y_market_ridge, test_size = 0.3, random_state = 2021)


In [119]:
#ridge_cv=RidgeCV(alphas=np.arange(2,,0.001))
ridge_cv=RidgeCV(alphas=np.arange(0.01,2,0.001))
model_cv=ridge_cv.fit(X_train,y_train)
model_cv.alpha_

1.9989999999999983

In [120]:
#refit model with alpha =3 and get score
rir = Ridge(alpha = model_cv.alpha_)
rir.fit(X_train,y_train)
y_pred_ridge = rir.predict(X_test)
MSE_ridge_market = mean_squared_error(y_test,y_pred_ridge)
MSE_ridge_market

4.3529755308054305

In [86]:
X_train.columns

Index(['crop_count', 'YEAR', 'HHsizemembers', 'HHsizeMAE', 'LandOwned',
       'LandCultivated', 'LivestockHoldings', 'NrofMonthsFoodInsecure',
       'score_HDDS_GoodSeason', 'score_HDDS_farmbasedGoodSeason',
       ...
       'Head_EducationLevel_religiousschool', 'Head_EducationLevel_secondary',
       'HFIAS_status_foodsecure', 'HFIAS_status_mildlyfi',
       'HFIAS_status_moderatelyfi', 'HFIAS_status_severelyfi',
       'continent_africa', 'continent_asia', 'continent_centralamerica',
       'continent_southamerica'],
      dtype='object', length=179)

In [ ]:
# -2.17617050e-02,  1.57421573e+00, -1.55559476e+00, -4.08594181e-03

In [165]:
rir.coef_

array([-5.62515887e-03, -2.77360355e-01,  2.75138952e-01, -1.34865111e-02,
        1.11426880e-02, -1.34214589e-01, -1.23223476e-02, -9.40756313e-03,
        1.19966975e-02, -9.21037960e-03, -2.76783889e-02, -1.82304145e-02,
       -1.24801190e-02,  8.78199219e-04,  9.17160765e-02,  1.53702998e-01,
       -9.24409434e-03, -1.46726095e-02, -1.62840499e-02, -1.83243486e-02,
       -1.42330557e-01,  8.96984350e-01,  1.91986252e-01, -1.95808023e-01,
       -2.27145710e-01, -1.53910088e-01, -1.61403036e-01,  4.64723907e-03,
       -8.53963210e-02,  8.53963210e-02,  0.00000000e+00, -5.27222230e-01,
        7.14951624e-03, -1.14079145e-01,  3.27231349e-02, -2.93132136e-02,
        6.37538028e-01,  0.00000000e+00, -4.20090001e-01,  9.93760982e-02,
       -3.28847625e-02,  1.60594530e-01,  5.47521030e-02, -3.23701315e-01,
       -2.59194047e-01,  1.73168642e-01, -7.66571770e-02, -2.48123653e-02,
        1.93312535e-01,  3.73063267e-01,  1.23197044e-01,  2.21050627e-01,
       -8.44205092e-02, -

In [125]:
feature_imp  = pd.DataFrame([X_train.columns, rir.coef_]).T

In [130]:
feature_imp = feature_imp.sort_values(by = 1,ascending = False).reset_index(drop = True)
feature_imp.iloc[0:15,:]

,0,1
0,livestock_prodsales_USD_PPP_pHH_Yr,0.896984
1,crop_name_1_broadbeans,0.637538
2,Head_EducationLevel_none,0.511744
3,HouseholdType_nonparenthead,0.511083
4,Head_EducationLevel_islamicschool,0.488995
5,HouseholdType_together,0.469186
6,Country_india,0.417186
7,Country_mali,0.41397
8,crop_name_1_custardapple,0.373063
9,Country_burkinafaso,0.30399


## Ridge regression with y=PPI_Likelihood

In [151]:
clean = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/STA_208/data/clean.csv")

In [152]:
from Functions import Cleaning_Functions
fun = Cleaning_Functions()

In [153]:
ppi_data = fun.delete_id_columns(clean) #1
ppi_data, pred_market = fun.drop_response_rows_with_NAs(ppi_data,"PPI_Likelihood" ,"Market_Orientation") #2
ppi_data = fun.replace_NAN_with_na(ppi_data) #3
ppi_data = fun.entry_to_lowercase(ppi_data) #4
ppi_data = fun.remove_underscores_spaces(ppi_data) #5
ppi_data = fun.convert_to_categorical(ppi_data) #6
ppi_data = fun.impute_data(ppi_data)#7

In [154]:
ppi_data.columns

Index(['crop_count', 'crop_name_1', 'crop_harvest_1', 'crop_intercrop_1',
       'YEAR', 'Country', 'HHsizemembers', 'HHsizeMAE', 'HouseholdType',
       'Head_EducationLevel', 'LandOwned', 'LandCultivated',
       'LivestockHoldings', 'NrofMonthsFoodInsecure', 'PPI_Likelihood',
       'HFIAS_status', 'score_HDDS_GoodSeason',
       'score_HDDS_farmbasedGoodSeason', 'score_HDDS_purchasedGoodSeason',
       'score_HDDS_BadSeason', 'score_HDDS_farmbasedBadSeason',
       'score_HDDS_purchasedBadSeason', 'TVA_USD_PPP_pmae_pday',
       'total_income_USD_PPP_pHH_Yr', 'offfarm_income_USD_PPP_pHH_Yr',
       'farm_income_USD_PPP_pHH_Yr', 'value_farm_produce_USD_PPP_pHH_Yr',
       'crop_sales_USD_PPP_pHH_Yr', 'value_crop_produce_USD_PPP_pHH_Yr',
       'value_crop_consumed_USD_PPP_pHH_Yr',
       'livestock_prodsales_USD_PPP_pHH_Yr',
       'value_livestock_production_USD_PPP_pHH_Yr',
       'value_livestock_prod_consumed_USD_PPP_pHH_Yr', 'Livestock_Orientation',
       'Food_Availability_kC

In [155]:
ppi_data.YEAR = ppi_data.YEAR.astype('category')
ppi_data = standarize_data(ppi_data)

In [157]:
X_ppi = get_dummyXs_y(ppi_data, "PPI_Likelihood")[0]
y_ppi = get_dummyXs_y(ppi_data, "PPI_Likelihood")[1]

In [158]:
X_train_ppi, X_test_ppi, y_train_ppi, y_test_ppi = train_test_split(X_ppi ,y_ppi, test_size = 0.3, random_state = 2021)


In [159]:
ppi_ridge_cv=RidgeCV(alphas=np.arange(0.01,2,0.001))
ppi_model_cv=ppi_ridge_cv.fit(X_train_ppi,y_train_ppi)
ppi_model_cv.alpha_

1.9989999999999983

In [160]:
#refit model with alpha =3 and get score
ridge_ppi = Ridge(alpha = ppi_model_cv.alpha_)
ridge_ppi.fit(X_train_ppi,y_train_ppi)
y_pred_ridge_ppi = ridge_ppi.predict(X_test_ppi)
MSE_ppi = np.mean((y_pred_ridge_ppi-np.array(y_test_ppi))**2)
MSE_ppi

0.49260599272351696

In [164]:
ridge_ppi.coef_

array([-2.17617050e-02,  1.57421573e+00, -1.55559476e+00, -4.08594181e-03,
       -1.06924612e-03,  2.18469444e-02,  1.02153683e-01, -9.42616691e-02,
       -2.24372851e-03, -1.57797529e-02, -5.88742124e-02, -1.79000079e-02,
        1.83649148e-02, -3.25600414e-01,  1.08376710e-01,  1.24168827e-01,
        4.00594059e-02,  4.08710064e-02,  4.01537011e-02,  4.01468648e-02,
       -1.06573761e-02, -1.15092269e-02,  3.74801606e-02,  4.63539810e-02,
       -4.58431018e-02,  5.90265287e-03, -1.02271205e-03,  2.52967978e-02,
        1.13778153e-02, -1.13778153e-02, -3.94688578e-01, -8.59401297e-02,
        0.00000000e+00,  1.26234693e-01, -4.35334739e-02,  6.72326473e-02,
       -1.72273060e-01,  0.00000000e+00,  1.69182864e-01, -4.29702313e-02,
       -2.01108918e-01, -1.66580489e-01,  6.63148726e-01, -1.79043274e-01,
       -4.25494823e-01, -3.29254485e-03, -1.45102680e-01, -5.74812424e-02,
       -2.35301350e-02,  1.06388091e-01,  1.62638927e-01, -3.45084404e-01,
        6.14391179e-02,  

In [166]:
feature_imp_ppi  = pd.DataFrame([X_train.columns, ridge_ppi.coef_]).T
feature_imp_ppi = feature_imp_ppi.sort_values(by = 1,ascending = False).reset_index(drop = True)
feature_imp_ppi.iloc[0:15,:]

,0,1
0,HHsizemembers,1.57422
1,Country_mali,1.33771
2,Country_honduras,1.23455
3,Country_burundi,1.09628
4,Country_guatemala,0.828848
5,crop_name_1_chilipepper,0.663149
6,crop_name_1_yam,0.47177
7,Head_EducationLevel_religiousschool,0.446823
8,crop_name_1_rambutan,0.44562
9,crop_name_1_passionfruit,0.437987


In [167]:
pd.concat([feature_imp_ppi, feature_imp],axis = 1).iloc[0:20,:]

,0,1,0,1
0,HHsizemembers,1.57422,livestock_prodsales_USD_PPP_pHH_Yr,0.896984
1,Country_mali,1.33771,crop_name_1_broadbeans,0.637538
2,Country_honduras,1.23455,Head_EducationLevel_none,0.511744
3,Country_burundi,1.09628,HouseholdType_nonparenthead,0.511083
4,Country_guatemala,0.828848,Head_EducationLevel_islamicschool,0.488995
5,crop_name_1_chilipepper,0.663149,HouseholdType_together,0.469186
6,crop_name_1_yam,0.47177,Country_india,0.417186
7,Head_EducationLevel_religiousschool,0.446823,Country_mali,0.41397
8,crop_name_1_rambutan,0.44562,crop_name_1_custardapple,0.373063
9,crop_name_1_passionfruit,0.437987,Country_burkinafaso,0.30399
